## Day 2 Exercise 2.1

(Beginner) For the enzymatic reaction system compare the results for the reduced and the full model and find the parameter regime in which the reduced model works. Solution available in Chapter 3 of [1].

In [1]:
# Biocrnpyler imports
from biocrnpyler.chemical_reaction_network import Species, Reaction, ChemicalReactionNetwork

# Math Packages
import numpy as np
from scipy.integrate import odeint

# Plotting packages
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

We are going to start by doing the **full model with massaction propensities.**

In [2]:
# Parameters
a = 1.0
d = 0.5
k_cat = 0.1

# Species
E = Species(name = "E", material_type = 'enzyme')
S = Species(name = "S", material_type = 'substrate')
C = Species(name = "C", material_type = 'complex')
P = Species(name = "P", material_type = 'product')

species = [E, S, C, P]

# Reactions
rxn1 = Reaction([E, S], [C], k = a, k_rev = d, propensity_type = 'massaction') # reversible reaction
rxn_cat = Reaction([C], [E, P], k = k_cat, propensity_type = 'massaction')

reactions = [rxn1, rxn_cat]

# Simulate the CRN
CRN1 = ChemicalReactionNetwork(species = species, reactions = reactions)
print('Full massaction model for enzymatic reaction system:\n',CRN1)

Full massaction model for enzymatic reaction system:
 Species = enzyme_E, substrate_S, complex_C, product_P
Reactions = [
	enzyme_E + substrate_S <--> complex_C        massaction: k_f(enzyme_E,substrate_S)=1.0*enzyme_E*substrate_S k_r(complex_C)=0.5*complex_C
	complex_C --> enzyme_E + product_P        massaction: k_f(complex_C)=0.1*complex_C
]


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:24: UserWarning: species which are formed of two species or more should be called using the chemical_reaction_network.ComplexSpecies constructor for attribute inheritance purposes.


In [3]:
timepoints = np.linspace(0, 100, 1000)
x_0 = {'enzyme_E':10, 'substrate_S':10, 'complex_C': 0, 'product_P':0}

R1 = CRN1.simulate_with_bioscrape(timepoints = timepoints, initial_condition_dict = x_0)

In [4]:
p = bokeh.plotting.figure(width = 550, height = 450, title = 'Full Model with massaction')
p.line(timepoints, R1['enzyme_E'], legend_label = 'E', color = 'blue')
p.line(timepoints, R1['complex_C'], legend_label = 'C', color = 'pink')
p.line(timepoints, R1['substrate_S'], legend_label = 'S', color = 'green')
p.line(timepoints, R1['product_P'], legend_label = 'P', color = 'red')

bokeh.io.show(p)

Now, we will look at the **reduced model with positive hill function propensities.**

In [5]:
# Parameters
K_X = 10.0
K_S = 1.0

# Species
E = Species(name = "E", material_type = 'enzyme')
S = Species(name = "S", material_type = 'substrate')
C = Species(name = "C", material_type = 'complex')
P = Species(name = "P", material_type = 'product')

species_red = [E, S, C, P]

# Reactions (same as before, change to hill positive with appropriate parameters)
rxn1_red = Reaction([E, S], [C], k = K_X, propensity_type = 'hillpositive',
                   propensity_params = {"s1":str(S), "K":K_S, "n":2})

rxn_cat_red = Reaction([C], [E, P], k = K_X, propensity_type = 'hillpositive',
                  propensity_params = {"s1":str(S), "K":K_S, "n":2})

reactions_red = [rxn1_red, rxn_cat_red]

# Simulate the CRN
CRN_red = ChemicalReactionNetwork(species = species_red, reactions = reactions_red)
print('Full massaction model for enzymatic reaction system:\n',CRN_red)

Full massaction model for enzymatic reaction system:
 Species = enzyme_E, substrate_S, complex_C, product_P
Reactions = [
	enzyme_E + substrate_S --> complex_C        hillpositive: k('substrate_S')=10.0*'substrate_S'^2/(1.0+'substrate_S'^2)
	complex_C --> enzyme_E + product_P        hillpositive: k('substrate_S')=10.0*'substrate_S'^2/(1.0+'substrate_S'^2)
]


/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-0.1-py3.7.egg/biocrnpyler/chemical_reaction_network.py:24: UserWarning: species which are formed of two species or more should be called using the chemical_reaction_network.ComplexSpecies constructor for attribute inheritance purposes.


In [6]:
timepoints = np.linspace(0, 100, 1000)
x_0_red = {'enzyme_E':10, 'substrate_S':10, 'complex_C': 0, 'product_P':0}

R1_red = CRN_red.simulate_with_bioscrape(timepoints = timepoints, initial_condition_dict = x_0_red)

In [7]:
p_red = bokeh.plotting.figure(width = 550, height = 450, title = 'Rediced Model with positive hill')
p_red.line(timepoints, R1['enzyme_E'], legend_label = 'E', color = 'blue')
p_red.line(timepoints, R1['complex_C'], legend_label = 'C', color = 'pink')
p_red.line(timepoints, R1['substrate_S'], legend_label = 'S', color = 'green')
p_red.line(timepoints, R1['product_P'], legend_label = 'P', color = 'red')

bokeh.io.show(p_red)